<h1 style="background-color:#FF7733;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
📐 תהליך הסבת שכבת חזיתות בתהליך לבנק"ל מודרני 📐 

In [ ]:
print(' Fronts In-Process:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import subprocess
import numpy as np
import pandas as pd
import polars as pl
import geopandas as gpd
from configs import CNFG
from threading import Thread
import dask_geopandas as daskgpd

<h1 style="text-align:right;font-size:100%">
העתקת שכבת מקור שכוללת גיאומטריה מקורית

In [ ]:
Thread(target = subprocess.call, args = [CNFG.BatchPath + "Fronts_InProcess-preprocess.bat"]).start();

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת חזיתות בתהליך קיימת

In [ ]:
fronts_cols = ['TEMP_ID', 'TALAR_NUM', 'TALAR_YEAR', 'ENTITY_STATE', 'FRONT_LEGAL_LENGTH', 'CUR_RADIUS', 'CADASTER_PROCESS', 'StartCoord', 'EndCoord']

fronts_inprocess = pl.read_csv(CNFG.CADSDE + 'Lines_InProcess.csv', columns = fronts_cols).to_pandas()

del fronts_cols
print('     input 1/4 read')

<h1 style="text-align:right;font-size:100%">
נתוני תהליכי קדסטר

In [ ]:
cpb_cols = ['ProcessGUID', 'ProcessName', 'ProcessType']

cpb_layer = pd.read_csv(CNFG.CSVPath + 'ProcessesAndRecords.csv', usecols = cpb_cols)

del cpb_cols

print('     input 2/4 read')

<h1 style="text-align:right;font-size:100%">
שמות תת"גים כפולים

In [ ]:
duplicated_names = pd.read_excel(CNFG.Local + 'tatag_duplicated_names.xlsx')['TatagName']
print('       Input 3/4 read')

<h1 style="text-align:right;font-size:100%">
נקודות בתהליך

In [ ]:
points_data = daskgpd.read_file(CNFG.ProductPath, layer = 'InProcessBorderPoints', columns = ['GlobalID', 'CPBUniqueID', 'geometry'], npartitions = 8).compute()

print('       Input 4/4 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('   Cleaning data')

<h1 style="text-align:right;font-size:100%">
חזיתות בעלות שטח רשום ריק יקבלו את הערך 0

In [ ]:
fronts_inprocess['FRONT_LEGAL_LENGTH'] = fronts_inprocess['FRONT_LEGAL_LENGTH'].fillna(0.0)

In [ ]:
print('   Data conversion')

<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות אורך מחושב ורדיוס

In [ ]:
fronts_inprocess.rename(columns = {'FRONT_LEGAL_LENGTH' : 'LegalLength', 'CUR_RADIUS' : 'Radius'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
סוג החזית יקבע לפי קיום של רדיוס

In [ ]:
fronts_inprocess['Radius'] = fronts_inprocess['Radius'].fillna(0)

straight = 1
curved   = 2
line_type_cond = fronts_inprocess['Radius'] == 0

fronts_inprocess['LineType'] = np.where(line_type_cond, straight, curved)

del [straight, curved, line_type_cond]

<h1 style="text-align:right;font-size:100%">
איחוד עם שכבת גבולות תהליכי קדסטר לקבלת מזהה של תהליך יוצר

In [ ]:
fronts_inprocess[['TALAR_NUM', 'TALAR_YEAR']] = fronts_inprocess[['TALAR_NUM', 'TALAR_YEAR']].fillna(0)
fronts_inprocess[['TALAR_NUM', 'TALAR_YEAR']] = fronts_inprocess[['TALAR_NUM', 'TALAR_YEAR']].astype(int)
fronts_inprocess['ProcessName'] = fronts_inprocess['TALAR_NUM'].astype(str) + '/' + fronts_inprocess['TALAR_YEAR'].astype(str)

duplicated_cond = (fronts_inprocess['ProcessName'].isin(duplicated_names)) & (fronts_inprocess['CADASTER_PROCESS'].isin([3,12])) #תת"ג
fronts_inprocess['ProcessName'] = np.where(duplicated_cond, '0' + fronts_inprocess['ProcessName'], fronts_inprocess['ProcessName'])

fronts_inprocess = fronts_inprocess.merge(right = cpb_layer, on = 'ProcessName', how = 'left')\
                                   .rename(columns = {'ProcessGUID' : 'CPBUniqueID'})\
                                   .drop(columns = ['TALAR_NUM', 'TALAR_YEAR', 'ProcessName', 'CADASTER_PROCESS'])

fronts_inprocess['CPBUniqueID'] = fronts_inprocess['CPBUniqueID'].fillna(CNFG.FabricatedCreateProcessGUID)


del [cpb_layer, duplicated_cond, duplicated_names]

<h1 style="text-align:right;font-size:100%">
ENTITY_STATE הגדרת סטאטוס לחזית בתהליך לפי שדה
<h1 style="text-align:right;font-size:100%">
חזיתות רקע מוסרות
<h1 style="text-align:right;font-size:100%">
חזיתות ללא ערך יוגדרו כביסוס
<h1 style="text-align:right;font-size:100%">
חזיתות שמגיעות מתהליכי תת"ג או קמ"ק מוגדרות כ-ביסוס וחדשה
<h1 style="text-align:right;font-size:100%">
חזיתות כפולות באותו תצ"ר יוגדרו כביסוס וחדשה וישארו כרשומה אחת

In [ ]:
fronts_inprocess = fronts_inprocess[fronts_inprocess['ENTITY_STATE'] != 2]

fronts_inprocess['ENTITY_STATE'] = fronts_inprocess['ENTITY_STATE'] + 1
fronts_inprocess.fillna(1, inplace=True)

fronts_inprocess['ENTITY_STATE'] = np.where(fronts_inprocess['ProcessType'].isin([4,5]), 3, fronts_inprocess['ENTITY_STATE'])


fronts_inprocess = fronts_inprocess.rename(columns = {'ENTITY_STATE' : 'LineStatus'})\
                                   .drop(columns = 'ProcessType')


duplicated = fronts_inprocess[fronts_inprocess.duplicated(['CPBUniqueID', 'StartCoord', 'EndCoord'], keep = False)]\
                             .sort_values(['CPBUniqueID'])\
                             .index.to_list()
                             
fronts_inprocess['LineStatus'] = np.where(fronts_inprocess.index.isin(duplicated), 3, fronts_inprocess['LineStatus'])
fronts_inprocess = fronts_inprocess.drop_duplicates(subset = ['CPBUniqueID', 'StartCoord', 'EndCoord', 'LineStatus'], keep = 'first', ignore_index = True)

del duplicated

<h1 style="text-align:right;font-size:100%">
ניתוח ואכלוס שדות למזה נקודת התחלה ומזהה נקודת סיום של חזית בתהליך לפי קואורדינטת התחלה וקואורדינטת סיום

In [ ]:
def reduce_point_resolution(point_series:pd.Series, decimals2keep:int = 2):
    
    point_series_x = str(point_series).split(',')[0]
    pre_decimal_x  = point_series_x.split('.')[0]
    post_decimal_x = point_series_x.split('.')[-1][:decimals2keep+1]
    reduced_x      = str(pre_decimal_x) + '.' + str(post_decimal_x)
    
    point_series_y = str(point_series).split(',')[-1]
    pre_decimal_y  = point_series_y.split('.')[0]
    post_decimal_y = point_series_y.split('.')[-1][:decimals2keep+1]
    reduced_y      = str(pre_decimal_y) + '.' + str(post_decimal_y)
    
    return str(reduced_x) + ',' + str(reduced_y)
    

points_data['Coord'] = points_data.geometry.x.astype(str) + ',' + points_data.geometry.y.astype(str)    
points_data['Coord'] = points_data['Coord'].apply(reduce_point_resolution)
points_data['key'] = points_data['Coord'] + ';' + points_data['CPBUniqueID']
points_data.drop(columns = 'geometry', inplace=True)


fronts_inprocess['StartCoord'] = fronts_inprocess['StartCoord'].apply(reduce_point_resolution)
fronts_inprocess['EndCoord'] = fronts_inprocess['EndCoord'].apply(reduce_point_resolution)

fronts_inprocess['StartCoord_key'] = fronts_inprocess['StartCoord'] + ';' + fronts_inprocess['CPBUniqueID']
fronts_inprocess['EndCoord_key'] = fronts_inprocess['EndCoord'] + ';' + fronts_inprocess['CPBUniqueID']

In [ ]:
x = fronts_inprocess.copy()

In [ ]:
fronts_inprocess = fronts_inprocess.merge(points_data, how = 'left', left_on = 'StartCoord_key', right_on = 'key')\
                                   .rename(columns = {'GlobalID':'StartPointUniqueID'})\
                                   .drop(columns = ['Coord', 'StartCoord', 'StartCoord_key', 'key'])

fronts_inprocess = fronts_inprocess.merge(points_data, how = 'left', left_on = 'EndCoord_key', right_on = 'key')\
                                   .rename(columns = {'GlobalID':'EndPointUniqueID'})\
                                   .drop(columns = ['Coord', 'EndCoord', 'EndCoord_key', 'key'])

fronts_inprocess['CPBUniqueID'] = fronts_inprocess['CPBUniqueID_x'].fillna(fronts_inprocess['CPBUniqueID_y'])
fronts_inprocess.drop(columns = ['CPBUniqueID_x', 'CPBUniqueID_y'], inplace = True)

del points_data

<h1 style="text-align:right;font-size:100%">
איכלוס שדות מזהה נקודת התחלה ומזהה נקודת סיום

In [ ]:
 print('   Defining new Layer')

<h1 style="text-align:right;font-size:100%">
הגדרת סוגי שדות נתונים

In [ ]:
fronts_inprocess = fronts_inprocess.astype({'LineType'           : int,
                                            'LineStatus'         : int,
                                            'Radius'             : float,
                                            'LegalLength'        : float,
                                            'CPBUniqueID'        : str,
                                            'StartPointUniqueID' : str,
                                            'EndPointUniqueID'   : str,
                                            'TEMP_ID'            : int})

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('   Exporting results to product folder')

In [ ]:
fronts_inprocess.to_csv(CNFG.CSVPath + 'InProcessFronts_data.csv', index = False)

In [ ]:
subprocess.call([CNFG.BatchPath + "Fronts In-Process-Save as Feature Class.bat"]);